In [0]:
!pip install optuna

In [0]:
import optuna
import torch

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [0]:
class LogisticNet(torch.nn.Module):
    def __init__(self, D_in, D_out):
        super(LogisticNet, self).__init__()
        self.linear = nn.Linear(D_in, D_out)

    def forward(self, x):
        lin = self.linear(x)
        return lin

In [0]:
def train(model, loss_func, optimizer, trX, trY):
    x = torch.Tensor(trX)
    y = torch.LongTensor(trY)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_func(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()
  
def valid(model, loss_func, valX, valY):
    x = torch.Tensor(valX)
    y = torch.LongTensor(valY)

    outputs = model(x)
    val_loss = loss_func(outputs, y)
    _, predY = torch.max(outputs.data, 1)
    correct = (predY == y.data).sum()
    val_acc = float(correct) / y.size(0)
    return val_loss.item(), val_acc

In [0]:
digits = load_digits()
data = digits['data']
target = digits['target']
# separate data

In [0]:
def sgd(model, trial, weight_decay):
    return optim.SGD(model.parameters(), lr=1e-2)
  
def adam(model, trial, weight_decay):
    return optim.Adam(model.parameters())

def create_optimizer(trial):
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam'])
    if optimizer[0] == 'sgd':
      return optim.SGD(model.parameters(), lr=1e-2)
    else:
      return optim.Adam(model.parameters())

In [0]:
def load_data():
  trX, teX, trY, teY = train_test_split(data, target, test_size=0.2, random_state=0)
  trX = torch.from_numpy(trX).float()
  teX = torch.from_numpy(teX).float()
  trY = torch.from_numpy(trY.astype(np.int64))
  teY = torch.from_numpy(teY.astype(np.int64))
  return (trX, trY, teX, teY)

In [0]:
def objective(trial):
  model = LogisticNet(64, 10)
  optimizer = create_optimizer(trial)
  loss_func = nn.CrossEntropyLoss()
  N_EPOCHS = 50
  for epoch in range(N_EPOCHS):
      loss = train(model, loss_func, optimizer, trX, trY)
      val_loss, val_acc = valid(model, loss_func, teX, teY)
  return 1 - val_acc

In [43]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)

[I 2019-03-09 03:37:10,971] Finished a trial resulted in value: 0.875. Current best value is 0.875 with parameters: {'optimizer': 'sgd'}.
[I 2019-03-09 03:37:11,085] Finished a trial resulted in value: 0.8555555555555556. Current best value is 0.8555555555555556 with parameters: {'optimizer': 'adam'}.
[I 2019-03-09 03:37:11,193] Finished a trial resulted in value: 0.8638888888888889. Current best value is 0.8555555555555556 with parameters: {'optimizer': 'adam'}.
[I 2019-03-09 03:37:11,299] Finished a trial resulted in value: 0.9555555555555556. Current best value is 0.8555555555555556 with parameters: {'optimizer': 'adam'}.
[I 2019-03-09 03:37:11,412] Finished a trial resulted in value: 0.8527777777777777. Current best value is 0.8527777777777777 with parameters: {'optimizer': 'adam'}.
[I 2019-03-09 03:37:11,525] Finished a trial resulted in value: 0.9222222222222223. Current best value is 0.8527777777777777 with parameters: {'optimizer': 'adam'}.
[I 2019-03-09 03:37:11,640] Finished 

In [45]:
print("best params: ", study.best_params)
print("best test accuracy: ", study.best_value)

best params:  {'optimizer': 'adam'}
best test accuracy:  0.8527777777777777
